#### 2. Bilingual Summarization and Translation :

Following the initial summarization, the same website content is going to be summarized again, this time with an added layer of complexity: translation into French.
The translated summary maintained the core message while ensuring linguistic and cultural nuances were respected.
This bilingual approach highlighted the adaptability of the LLMs in handling multilingual tasks effectively


In [1]:
# IMPORT LABRAIRIES

import requests
import logging
from typing import Optional, Tuple

# CCONFIGURE LOGGING
logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger(__name__)

# CREATE CONSTANTS
OLLAMA_API_URL = 'http://localhost:11434/api/generate'
DEFAULT_MODEL = 'llama3'  # Can be changed to llama3 or other available models
SUMMARY_MAX_TOKENS = 350  # Increased for detailed bullet points
TRANSLATION_MAX_TOKENS = 400  # French translations often require more tokens

In [2]:
# CREATE FUNCTIONS

def call_ollama_api(prompt: str, model: str = DEFAULT_MODEL, 
                   max_tokens: int = SUMMARY_MAX_TOKENS, 
                   temperature: float = 0.5) -> Optional[str]:
    """
    Calls Ollama API with the given prompt and returns the response.
    No timeout to accommodate variable processing times.
    """
    try:
        payload = {
            'model': model,
            'prompt': prompt,
            'max_tokens': max_tokens,
            'temperature': temperature,
            'stream': False
        }
        
        response = requests.post(OLLAMA_API_URL, json=payload)
        response.raise_for_status()
        
        return response.json().get('response', '').strip()
    
    except requests.exceptions.RequestException as e:
        logger.error(f"API Error: {str(e)}")
    except Exception as e:
        logger.error(f"Unexpected Error: {str(e)}")
    return None

def generate_structured_summary(url: str) -> Tuple[Optional[str], Optional[str]]:
    """
    Generates a structured English summary and French translation of a website.
    Returns tuple of (english_summary, french_translation)
    """
    # English prompt with strict formatting requirements
    english_prompt = f"""
    Analyze the content from {url} and provide a summary in 4-5 bullet points covering:
    1. The company's core purpose/mission
    2. Their main services or products
    3. Key value propositions/differentiators
    4. Primary target audience/customers
    
    Requirements:
    - Use concise bullet point format
    - Include only explicitly stated facts from the content
    - Omit any speculative or interpretive statements
    - Each bullet point should be 1-2 sentences maximum
    - Maintain professional tone
    
    Structure your response like this:
    • [Purpose]: ...
    • [Services]: ...
    • [Value]: ...
    • [Audience]: ...
    """
    
    # Generate English summary
    english_summary = call_ollama_api(english_prompt, max_tokens=SUMMARY_MAX_TOKENS)
    if not english_summary:
        return None, None
    
    # French translation prompt
    french_prompt = f"""
    Translate the following business summary from English to French:
    {english_summary}
    
    Requirements:
    - Maintain all bullet point formatting
    - Preserve professional business tone
    - Keep technical/business terms accurate
    - Ensure complete grammatical correctness
    """
    
    # Generate French translation
    french_translation = call_ollama_api(french_prompt, max_tokens=TRANSLATION_MAX_TOKENS)
    
    return english_summary, french_translation or "Translation failed"

def display_results(english: str, french: str) -> None:
    """Formats and displays the results clearly"""
    divider = "=" * 50
    logger.info("\n%s\nENGLISH SUMMARY:\n%s\n", divider, english)
    logger.info("\n%s\nFRENCH TRANSLATION:\n%s\n%s\n", divider, french, divider)

if __name__ == "__main__":
    website_url = "https://kamatechsolutions.com"
    
    logger.info("Generating structured summary and translation...")
    summary, translation = generate_structured_summary(website_url)
    
    if summary:
        display_results(summary, translation)
    else:
        logger.error("Failed to generate summary")

Generating structured summary and translation...

ENGLISH SUMMARY:
Here is the summary based on the content from https://kamatechsolutions.com:

• **Core Purpose/Mission**: Kamatech Solutions aims to provide innovative and cost-effective IT solutions, enabling businesses to streamline their operations and improve overall efficiency.

• **Main Services/Products**: The company offers a range of services including software development, IT consulting, cybersecurity, data analytics, and cloud solutions. They also have a portfolio of proprietary products for industries such as healthcare, finance, and government.

• **Key Value Propositions/Differentiators**: Kamatech Solutions emphasizes its expertise in understanding clients' specific needs, providing tailored solutions, and ensuring seamless integration with existing systems. They also highlight their commitment to security, scalability, and reliability.

• **Primary Target Audience/Clients**: The company's target audience appears to be s

### **leveraging api/chat Endpoint**
Using **/api/chat** instead of **/api/generate** better leverages Ollama's chat capibilities while maintenaing the original requirements for structured business summaries and translations. 

When working with **LLM APIs**, the choice between **/api/chat** and **/api/generate endpoints** depends on your specific use case:


In [3]:
import requests
import logging
from typing import Optional, Tuple

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger(__name__)

# Constants
OLLAMA_API_URL = 'http://localhost:11434/api/chat'  # Using chat endpoint
DEFAULT_MODEL = 'llama3'  # Can be changed to mistral or other available models

def call_ollama_chat(messages: list, model: str = DEFAULT_MODEL) -> Optional[str]:
    """
    Calls Ollama's chat endpoint with conversation history
    """
    try:
        payload = {
            'model': model,
            'messages': messages,
            'stream': False
        }
        
        response = requests.post(OLLAMA_API_URL, json=payload)
        response.raise_for_status()
        
        return response.json()['message']['content'].strip()
    
    except requests.exceptions.RequestException as e:
        logger.error(f"API Error: {str(e)}")
    except Exception as e:
        logger.error(f"Unexpected Error: {str(e)}")
    return None

def generate_structured_summary(url: str) -> Tuple[Optional[str], Optional[str]]:
    """
    Generates structured summary and translation using chat endpoint
    """
    # System message to set behavior
    system_msg = {
        'role': 'system',
        'content': "You are a business analyst that extracts and summarizes information with perfect accuracy."
    }
    
    # English summary prompt
    english_prompt = {
        'role': 'user',
        'content': f"""Analyze {url} and provide a summary in this exact format:
        
        • [Purpose]: <company's core mission>
        • [Services]: <main offerings>
        • [Value]: <key differentiators>
        • [Audience]: <target customers>
        
        Rules:
        - Use only explicitly stated facts
        - No interpretation or speculation
        - Maximum 2 sentences per bullet
        - Maintain professional tone"""
    }
    
    # Generate English summary
    english_summary = call_ollama_chat([system_msg, english_prompt])
    if not english_summary:
        return None, None
    
    # French translation prompt
    french_prompt = {
        'role': 'user',
        'content': f"""Translate this business summary to French while preserving:
        
        {english_summary}
        
        Requirements:
        - Keep bullet point format
        - Maintain technical/business terms
        - Ensure perfect grammar
        - Use professional tone"""
    }
    
    # Generate French translation
    french_translation = call_ollama_chat([system_msg, french_prompt])
    
    return english_summary, french_translation or "Translation failed"

def display_results(english: str, french: str) -> None:
    """Formats and displays results"""
    divider = "=" * 60
    logger.info("\n%s\nENGLISH SUMMARY:\n%s\n", divider, english)
    logger.info("\n%s\nFRENCH TRANSLATION:\n%s\n%s\n", divider, french, divider)

if __name__ == "__main__":
    website_url = "https://kamatechsolutions.com"
    
    logger.info("Generating business summary and translation...")
    summary, translation = generate_structured_summary(website_url)
    
    if summary:
        display_results(summary, translation)
    else:
        logger.error("Failed to generate summary")

Generating business summary and translation...

ENGLISH SUMMARY:
Based on the provided website, here is the summary:

• [Purpose]: Kamatechsolutions.com aims to provide innovative technology solutions for businesses and organizations, enabling them to streamline their operations and improve overall efficiency.
• [Services]: The company offers a range of services including custom software development, consulting, data analytics, and cybersecurity solutions. They also provide training and support to ensure seamless integration with existing systems.

• [Value]: Kamatechsolutions.com emphasizes its ability to understand client needs and develop tailored solutions to meet those requirements. Their team of experts is committed to delivering high-quality results while maintaining a focus on customer satisfaction.

• [Audience]: The company's target audience appears to be small to medium-sized businesses, as well as organizations in various industries looking for technology-based solutions to

----------------------------------------------------------------------------------------------------------------

### Key takeaways :
- The model uses **[llama3](https://ollama.com/search)** which is the most capable model openly model available model LLM to date and one of the most popular open-source models available in **Ollama**.
- No API Key is needed: **Ollama** runs locally, so no API Key is required.
- Customization: You can adjust the max_tokens and temperature parameters to control the response lengh and creativity.

### /api/chat vs /api/generate :
- **api/chat :**
  - Best for conversational applications where context and history matter;
  - Designed for back-and-forth conversation that build on previous messages or chats;
  - Typically supports systems instructions, user messages, and assistant responses;
  - Usually returns more natural-sounding responses appriopriate for dialogue.

- **api/generate :**
    - Better for one-time text generation without chat or conversation history;
    - Often used for context creation or generation, completion tasks, or summarization;
    - Usually simple structure with only a prompt and completion;
    - May provide greater control over specific generation parameters.
  